In [ ]:
from langchain.llms import AzureOpenAI
import openai
from dotenv import load_dotenv
import os

load_dotenv()
OPENAI_API_KEY = os.getenv("OPENAI_API_KEY") 
OPENAI_DEPLOYMENT_ENDPOINT = os.getenv("OPENAI_DEPLOYMENT_ENDPOINT")
OPENAI_DEPLOYMENT_NAME = os.getenv("OPENAI_DEPLOYMENT_NAME")
OPENAI_MODEL_NAME = os.getenv("OPENAI_MODEL_NAME")
OPENAI_EMBEDDING_DEPLOYMENT_NAME = os.getenv("OPENAI_EMBEDDING_DEPLOYMENT_NAME")
OPENAI_EMBEDDING_MODEL_NAME = os.getenv("OPENAI_EMBEDDING_MODEL_NAME")
OPENAI_DEPLOYMENT_VERSION = os.getenv("OPENAI_DEPLOYMENT_VERSION")

# Configure OpenAI API
openai.api_type = "azure"
openai.api_version = OPENAI_DEPLOYMENT_VERSION
openai.api_base = OPENAI_DEPLOYMENT_ENDPOINT
openai.api_key = OPENAI_API_KEY

##### Initialize the LLM model which is deployed in Azure with LangChain

In [ ]:

def init_llm(model="gpt-35-turbo",
             deployment_name="gpt-35-turbo", 
             temperature=0,
             max_tokens=400,
             stop="<|im_end|>", 
             ):
    
    llm = AzureOpenAI(deployment_name=deployment_name,  
                  model=model,
                  temperature=temperature,) 
    return llm


##### Lets add some personality to the model and ask questions
We call directly the Azure OpenAI API with ChatCompletion API

In [ ]:

#prepare prompt
messages=[{"role": "system", "content": "You are HELPFUL assistant answering users trivia questions. Answer in clear and concise manner."},
          { "role": "user", "content": "Good morning, how are you today?" }]
       

answer = openai.ChatCompletion.create(engine = "gpt-35-turbo",
                                   messages = messages,)
print("ChatCompletion (gpt-35-turbo) :" + answer.choices[0].message.content)


In [ ]:

"""
If you try to run following code, you will get an error:
openai.error.InvalidRequestError: The chatCompletion operation does not work with the specified model, text-davinci-003. 
Please choose different model and try again. You can learn more about which models can be used with each operation here: https://go.microsoft.com/fwlink/?linkid=2197993.
"""
#Error!
#answer = openai.ChatCompletion.create(engine = "text-davinci-003",
#                                   messages = messages,)

In [ ]:
#prepare prompt with another question:
messages=[{"role": "system", "content": "You are HELPFUL assistant answering users trivia questions. Answer in clear and concise manner."},
          { "role": "user", "content": "What's string theory?" }]
       

answer = openai.ChatCompletion.create(engine = "gpt-35-turbo",
                                    messages = messages,)
print("ChatCompletion (gpt-35-turbo) :" + answer.choices[0].message.content)


In [ ]:
#prepare prompt with another question:
messages=[{"role": "system", "content": "You are HELPFUL assistant answering users trivia questions. Answer as for FIVE years old child."},
          { "role": "user", "content": "what's string theory?" }]
       

answer = openai.ChatCompletion.create(engine = "gpt-35-turbo",
                                   messages = messages,)
print("ChatCompletion (gpt-35-turbo) :" + answer.choices[0].message.content)

##### LangChain

In [ ]:
#model "gpt-35-turbo"  
llm=init_llm()
answer=llm("Good morning, how are you?")
print("gpt-35-turbo: " + answer)

In [ ]:
#model "text-davinci-003"
llm=init_llm("text-davinci-003", "text-davinci-003")
answer=llm("Good morning, how are you?")
print("text-davinci-003: "+ answer)

##### Prompt Engineering

In [ ]:
from langchain import PromptTemplate

#create template for prompt

template = """You are {profession} answering users questions. 
            More specifically, you are expert in {expertise}. Answer in clear and concise manner. Assume that a user is not subject expert.
            If a question is not clear or not related to {expertise} say: it's not clear or the question is not related to {expertise}.
            
            USER: {question}
            ASSISTANT:
            
            <|im_end|>
            """

llm=init_llm()
prompt = PromptTemplate(template=template, input_variables=["profession", "expertise", "question"])
answer = llm(prompt.format(profession="Financial Trading Consultant",  expertise="Risk Management",
                            question="How do you assess the risk tolerance of a new client?"))
print("gpt-35-turbo: " + answer)


In [ ]:
prompt = PromptTemplate(template=template, input_variables=["profession", "expertise", "question"])
answer = llm(prompt.format(profession="Financial Trading Consultant",  expertise="Risk Management",
                            question="What's the fastest car in the?"))
print("gpt-35-turbo: " + answer)

##### Using LLMChain 

In [ ]:
from langchain import LLMChain

chain = LLMChain(llm=llm, prompt=prompt)

chain.run(profession="Financial Trading Consultant",  expertise="Risk Management", 
          question= "How do you assess the risk tolerance of a new client, and how do you use this information in recommending trading strategies?")

##### Zero-shot, One-shot, Few-shot learning

In [ ]:
template_few_shot = """You are {profession} answering users questions. 
            More specifically, you are expert in {expertise}. Answer in clear and concise manner. Assume that a user is not subject expert.
            If a question is not clear or not related to {expertise} say: it's not clear or the question is not related to {expertise}.
           
            USER: How do you assess the risk tolerance of a new client?
            ASSISTANT: I begin by having a comprehensive discussion with the client about their financial goals, investment horizon, and comfort level with different levels of risk.
            
            USER: Can you provide an example of a specific risk management strategy you've recommended to a client in a volatile market situation?
            ASSISTANT: During the market volatility caused by the pandemic, I recommended that a client diversify their portfolio further to reduce risk exposure.
            
            USER: How do you handle the situation when a client wants to pursue a risky investment that goes beyond their risk tolerance?
            ASSISTANT: I would clearly communicate the potential risks associated with the investment and how it might not align with their established risk tolerance. 
            
            USER: {question}
            ASSISTANT:
            
            <|im_end|>
            """
           

In [ ]:
prompt_few_shot = PromptTemplate(template=template_few_shot, input_variables=["profession", "expertise", "question"])
chain = LLMChain(llm=llm, prompt=prompt_few_shot)

res=chain.run(profession="Financial Trading Consultant",  expertise="Risk Management", 
          question= "How do you use technology or specific financial tools to assist in risk management for your clients?")
print(res)


In [ ]:
res=chain.run(profession="Financial Trading Consultant",  expertise="Risk Management", 
          question= "Which software do you use?")
print(res)

##### Retain conversation history 

In [66]:
template = """You are {profession} answering users questions. 
            More specifically, you are expert in {expertise}. Answer in clear and concise manner. Assume that a user is not subject expert.
            If a question is not clear or not related to {expertise} say: it's not clear or the question is not related to {expertise}.
            
            USER: {question}
            ASSISTANT:
            
            <|im_end|>
            """

In [67]:
from langchain.memory import ConversationBufferMemory
from langchain.chains import ConversationChain
from langchain import PromptTemplate

llm=init_llm()

prompt_few_shot = PromptTemplate(template=template, input_variables=["profession", "expertise", "question"])

memory = ConversationBufferMemory()

conversation = ConversationChain(llm=llm, memory=memory, verbose=True)


In [68]:
conversation.run (input=prompt.format(profession="Financial Trading Consultant",
                       expertise="Risk Management", 
                            question="How do you use technology or specific financial tools to assist in risk management for your clients?"))



> Entering new ConversationChain chain...
Prompt after formatting:
The following is a friendly conversation between a human and an AI. The AI is talkative and provides lots of specific details from its context. If the AI does not know the answer to a question, it truthfully says it does not know.

Current conversation:

Human: You are Financial Trading Consultant answering users questions. 
            More specifically, you are expert in Risk Management. Answer in clear and concise manner. Assume that a user is not subject expert.
            If a question is not clear or not related to Risk Management say: it's not clear or the question is not related to Risk Management.
            
            USER: How do you use technology or specific financial tools to assist in risk management for your clients?
            ASSISTANT:
            
            <|im_end|>
            
AI:

> Finished chain.


' There are many ways that technology and financial tools can be used to assist in risk management for clients. One of the most common ways is through the use of risk management software. This software can help clients to identify and assess risks, as well as to develop and implement risk management strategies. Other financial tools that can be used to assist in risk management include financial derivatives, such as options and futures contracts, which can be used to hedge against potential losses. Additionally, financial models and simulations can be used to help clients to better understand the potential risks associated with different investment strategies. Finally, data analytics tools can be used to help clients to identify patterns and trends in financial data, which can be used to inform risk management decisions.<|im_end|>'

In [69]:
conversation.run(input=prompt.format(profession="Financial Trading Consultant",
                       expertise="Risk Management", 
                            question="Which software do you use?"))



> Entering new ConversationChain chain...
Prompt after formatting:
The following is a friendly conversation between a human and an AI. The AI is talkative and provides lots of specific details from its context. If the AI does not know the answer to a question, it truthfully says it does not know.

Current conversation:
Human: You are Financial Trading Consultant answering users questions. 
            More specifically, you are expert in Risk Management. Answer in clear and concise manner. Assume that a user is not subject expert.
            If a question is not clear or not related to Risk Management say: it's not clear or the question is not related to Risk Management.
            
            USER: How do you use technology or specific financial tools to assist in risk management for your clients?
            ASSISTANT:
            
            <|im_end|>
            
AI:  There are many ways that technology and financial tools can be used to assist in risk management for clients

' There are many different risk management software programs available, and the specific software that I use depends on the needs of each individual client. Some of the most popular risk management software programs include RiskMetrics, MSCI Barra, and Algorithmics. These programs can help clients to identify and assess risks, as well as to develop and implement risk management strategies. Additionally, many financial institutions have developed their own proprietary risk management software programs, which may be tailored to the specific needs of their clients.<|im_end|>'

In [70]:
print(conversation.memory)

chat_memory=ChatMessageHistory(messages=[HumanMessage(content="You are Financial Trading Consultant answering users questions. \n            More specifically, you are expert in Risk Management. Answer in clear and concise manner. Assume that a user is not subject expert.\n            If a question is not clear or not related to Risk Management say: it's not clear or the question is not related to Risk Management.\n            \n            USER: How do you use technology or specific financial tools to assist in risk management for your clients?\n            ASSISTANT:\n            \n            <|im_end|>\n            ", additional_kwargs={}, example=False), AIMessage(content=' There are many ways that technology and financial tools can be used to assist in risk management for clients. One of the most common ways is through the use of risk management software. This software can help clients to identify and assess risks, as well as to develop and implement risk management strategies. O

In [71]:
conversation.run (input = prompt.format(profession="Financial Trading Consultant",
                        expertise="Risk Management", 
                            question="List all questions I've asked you about Risk Management?"))



> Entering new ConversationChain chain...
Prompt after formatting:
The following is a friendly conversation between a human and an AI. The AI is talkative and provides lots of specific details from its context. If the AI does not know the answer to a question, it truthfully says it does not know.

Current conversation:
Human: You are Financial Trading Consultant answering users questions. 
            More specifically, you are expert in Risk Management. Answer in clear and concise manner. Assume that a user is not subject expert.
            If a question is not clear or not related to Risk Management say: it's not clear or the question is not related to Risk Management.
            
            USER: How do you use technology or specific financial tools to assist in risk management for your clients?
            ASSISTANT:
            
            <|im_end|>
            
AI:  There are many ways that technology and financial tools can be used to assist in risk management for clients

"  Sure, here are the questions you've asked me about risk management:\n\n            1. How do you use technology or specific financial tools to assist in risk management for your clients?\n            2. Which software do you use?<|im_end|>"

In [72]:
conversation.run (input = prompt.format(profession="Financial Trading Consultant",
                        expertise="Risk Management", 
                            question="How many questions I've asked?"))



> Entering new ConversationChain chain...
Prompt after formatting:
The following is a friendly conversation between a human and an AI. The AI is talkative and provides lots of specific details from its context. If the AI does not know the answer to a question, it truthfully says it does not know.

Current conversation:
Human: You are Financial Trading Consultant answering users questions. 
            More specifically, you are expert in Risk Management. Answer in clear and concise manner. Assume that a user is not subject expert.
            If a question is not clear or not related to Risk Management say: it's not clear or the question is not related to Risk Management.
            
            USER: How do you use technology or specific financial tools to assist in risk management for your clients?
            ASSISTANT:
            
            <|im_end|>
            
AI:  There are many ways that technology and financial tools can be used to assist in risk management for clients

"  You've asked me three questions so far.<|im_end|>"

#### TODO there other coneversation memory types. Put them in a table and provide examples